In [1]:
import gurobipy as gp
import numpy as np
import math
import pandas as pd 
from gurobipy import GRB

In [2]:
df1 = pd.read_excel('ExamRequirements_0326.xlsx') # Exam Information
df2 = pd.read_excel('StudentExam_0326.xlsx') # Student Information

FileNotFoundError: [Errno 2] No such file or directory: 'ExamRequirements_0326.xlsx'

In [ ]:

Course_Duration = dict(zip(df1['Course Code'], df1['Duration']))
for course in Course_Duration:
    Course_Duration[course] = int(Course_Duration[course].strip('Minutes'))
Courses, Duration = gp.multidict(Course_Duration)


In [ ]:

Course_Duration

In [ ]:
ID_Course = df2.groupby('SID')['Course Code'].apply(set).to_dict() 
Course_ID = df2.groupby('Course Code')['SID'].apply(set).to_dict()


In [ ]:
Course_ID["MED2040"] & Course_ID["PHM2002"]


In [ ]:
NumberofCourses = len(Courses)
Timeslots = range(24)# everyday has three time slots 8:30, 13:30, 18:30(totally 3*8 slots)
CourseCapacity = {i:len(Course_ID[i]) for i in Courses}
Sharing = dict()
for i in Courses:
    for j in Courses:
        if i == j:
            Sharing[i,j] = 0
        else:
            Sharing[i,j] = len(Course_ID[i] & Course_ID[j])
CoursePair, Sharing = gp.multidict(Sharing)


Small_Courses = [course for course in Courses if CourseCapacity[course]<=100]



In [ ]:
count = 0 
for i in Sharing :
    if Sharing[i] > 0 :
        count += 1
count

In [ ]:

TriSharing = dict()
for i in Courses:
    for j in Courses:
        for k in Courses:
            if (i == j) or (i == k) or (j == k):
                TriSharing[i,j,k] = 0
            else:
                TriSharing[i,j,k] = len(Course_ID[i] & Course_ID[j]& Course_ID[k])
TriCourse, TriSharing = gp.multidict(TriSharing)

In [ ]:
t_830 = [0,3,6,9,12,15,18,21]
t_1330 = [i+1 for i in t_830]
t_1830 = [i+2 for i in t_830]
d1 = [0,1,2]
d2 = [3,4,5]
d3 = [6,7,8]
d4 = [9,10,11]
d5 = [12,13,14]
d6 = [15,16,17]
d7 = [18,19,20]
d8 = [21,22,23]


In [ ]:
exam = gp.Model()
# Here we set the time limit as 20 min or 30 min to lower the gap. 
# totally we have 42w constrains, which takes us a long time to solve it.
exam.params.timelimit=10*60


start_time = exam.addVars(Courses,Timeslots,vtype='B',name='start')
robust_variable = exam.addVar(vtype='C',name='robust')
# y = exam.addVars(Courses,lb=0,name='y')

small_course_robust = exam.addVar(vtype='C',name='small_robust')

In [ ]:
def special_constraint(p_course, p_candidate, p_suggest):
    l_sum = gp.LinExpr()
    for k in p_candidate:
        l_sum += start_time[p_course,k]
    if p_suggest == True:
        # exam.addConstr(gp.quicksum(start_time[p_course,k]==1 for k in p_candidate))

        exam.addConstr(l_sum==1)
    else:
        # exam.addConstr(gp.quicksum(start_time[p_course,k]==0 for k in p_candidate))
        exam.addConstr(l_sum==0)
    exam.update()

# the sum of exams in the weekend 
tail_sum = gp.LinExpr()
for k in d1+d2+d7+d8:
    for course in Courses:
        tail_sum += start_time[course,k]*CourseCapacity[course]
# make this sum small ASAP

small_course_sum = dict()
for k in Timeslots:
    small_course_sum[k] = gp.LinExpr()
    for course in Small_Courses:
            small_course_sum[k] += start_time[course,k]*CourseCapacity[course]

In [ ]:
# No three successive exams 
for i in Courses:
    for j in Courses:
        for p in Courses:
            if TriSharing[i,j,p]>0:
                for k in t_830:
                    exam.addConstr(start_time[i,k]+start_time[j,k+1]+start_time[p,k+2]<=2)

In [ ]:
exam.addConstrs(start_time.sum(i,'*')==1 for i in Courses)

# add constrain that no exam on weekend night
for i in Courses :
    exam.addConstr(start_time[i,2] + start_time[i,5] + start_time[i,20] + start_time[i,23]  == 0)

# no exams overlaping
for i in Courses:
    for j in Courses:
        if Sharing[i,j]>0:
            for k in Timeslots:
                exam.addConstr(start_time[i,k]+start_time[j,k]<=1)


# maximum of student number is 1400 in one day
exam.addConstrs(gp.quicksum(start_time[i,j]*CourseCapacity[i] for i in Courses)<=1400 for j in Timeslots)
# exam.addConstrs(gp.quicksum(start_time[i,j]*CourseCapacity[i] for i in Courses) >=500 for j in [0,1])
# maximum of student number is 500 in the night
exam.addConstrs(gp.quicksum(start_time[i,3*j+2]*CourseCapacity[i] for i in Courses)<=500 for j in range(8))

#  maximum of student number is 500 in the last day
exam.addConstrs(gp.quicksum(start_time[i,j]*CourseCapacity[i] for i in Courses)<= 200 for j in d8)


exam.addConstrs(small_course_sum[k] <= small_course_robust for k in Timeslots)


# for course in Courses:
#     if CourseCapacity[course]>=150:
#         special_constraint(course,d1+d8,False)


# some special constrains     

# Course requirement
special_constraint("PHY1001", d1+d2+d3+d4+d5+d6, True)# Before 17th
special_constraint("FIN2010", d1+d2, True) # First two days
special_constraint("MAT2040", d5+d6, True) # in 15th or 16th
special_constraint('ECO3121', d1+d2+d3, True) #ASAP
special_constraint('BIO1001', [6], True) # 13th morning
special_constraint('MED2002', [0], True) # 11TH MAY 8:30-11:30 
special_constraint('MED2012', [3], True) # 12TH MAY 8:30-11:30
special_constraint('MED2040', [9], True) # 14TH MAY 8:30-10:30
special_constraint('MED2060', [6], True) # 13TH MAY 8:30-10:30
special_constraint('BIO2004', [12], True) # 15 MAY, IN THE MORNING
special_constraint('MED2030', [12], True) # 15TH MAY 8:30-10:30
special_constraint('MED2100', [13], True) # 15TH MAY 14:00-15:00
special_constraint('MED1002', [3], True) # 12TH MAY 9:00-12:00 
special_constraint('MED1012', [0], True) # 11TH MAY 9:00-12:00
special_constraint('MED1022', [6], True) # 13TH MAY 9:00-11:00
special_constraint('MED1032', [9], True)# 14TH MAY 9:00-11:00
special_constraint('PHM2002', [9],True) # 14th 8:30-10:30
                   



# Scheduling requirement
special_constraint('MAT1002',[0,1,3,4]+d3,True)#ASAP 
special_constraint('PHY1001',[0,1,3,4]+d3,True)#ASAP 
special_constraint('FIN2010',[0,1,3,4]+d3,True)#ASAP
special_constraint('DMS2030',[0,1,3,4]+d3,True)#ASAP  
special_constraint('ECO2121',[0,1,3,4]+d3,True)#ASAP 

# CSJ special constrains     
special_constraint("FIN2010", [1,2]+d1, True) # 11TH MAY 13:30
special_constraint("DDA4210", d3, True) # 13TH MAY 18:30
# special_constraint("DMS2030", d4, True) # 14TH MAY 13:30
special_constraint("STA3005", d5+d6+d7, True) # 15TH MAY 13:30
special_constraint("CSC3100", d5+d6+d7, True) # 17TH MAY 13:30




# exam.addConstr(evening_sum==0)
# exam.addConstrs(slot_sum[k]<=800 for k in Timeslots)
# exam.addConstrs(start_time.sum('*',j)>= robust_variable for j in Timeslots)
# exam.setObjective(10*robust_variable-1/8*tail_sum,sense=GRB.MAXIMIZE)
exam.setObjective(1/2*tail_sum+small_course_robust,sense=GRB.MINIMIZE)
exam.optimize()

In [ ]:
def busy(schedule):
    if len(schedule)==1:
        return False
    elif len(schedule)==2:
        return schedule[1]-schedule[0]<=1
    else:
        for i in range(1,len(schedule)-1):
            if schedule[i]-schedule[i-1]==1 and schedule[i+1]-schedule[i]==1:
                return True

In [ ]:
Schedule = {}
for i in Courses:
    for j in Timeslots:
        if exam.getVarByName('start[{},{}]'.format(i,j)).X>0:
            Schedule[i]=j

In [ ]:
len(ID_Course)

In [ ]:
small_course_robust.X

In [ ]:
# check the rate of the non-zero terms

count = 0
nonzero = 0
for i in TriCourse :
    count += 1
    if TriSharing[i] > 0 :
        nonzero += 1

# totally we have 778688 triple tumples and alomst 14418 of them has value bigger than 1
# the rate is 1.8%

In [ ]:
count

In [ ]:
nonzero

In [ ]:
tail_sum.getValue()

In [ ]:
day1 =  [0,1,2]
day2 =  [3,4,5]
day3 =  [6,7,8]
day4 =  [9,10,11]
day5 =  [12,13,14]
day6 =  [15,16,17]
day7 =  [18,19,20]
day8 =  [21,22,23]
days = [day1,day2,day3,day4,day5,day6,day7,day8]
def busy2(Schedule):
    for day in days:
        if len(set(Schedule) & set(day))>=3:
            return True
    else:
        return False

In [ ]:
StudentSchedule = {}
for student in ID_Course:
    temp = list()
    for course in ID_Course[student]:
        temp.append(Schedule[course])
    StudentSchedule[student]=sorted(temp)
counter = 0
for student in StudentSchedule:
    if busy(StudentSchedule[student]):
        counter+=1
        
        # print(student,ID_Course[student],StudentSchedule[student])
counter1 = 0
for student in StudentSchedule:
    i = StudentSchedule[student]
    if busy2(i):
        counter1 +=1
        print(student,i,ID_Course[student])
# showing the consecutive exams of student 
counter1 

In [ ]:
counter

In [ ]:
for student in ID_Course:
    if len(ID_Course[student])>=5:
        print(ID_Course[student])

In [ ]:
for j in Timeslots:
    count = 0
    for i in Courses:
        count += start_time[i,j].X
    print(count)

In [ ]:
[gp.quicksum(start_time[i,j].X*CourseCapacity[i] for i in Courses) for j in Timeslots]

In [ ]:
exam.write('exam(gurobi).mps')

In [ ]:
import xlwt
wb= xlwt.Workbook(encoding='uts-8')
# set the style of the excel
style = xlwt.XFStyle()#the label (first row)
style1 = xlwt.XFStyle()#the even days
style2 = xlwt.XFStyle()#the odd days

font_label = xlwt.Font()
font_label.name= "Calibri"
font_label.bold = True
font_label.height = 240

font = xlwt.Font()
font.name= "Calibri"
font.bold = False
font.height = 240

border = xlwt.Borders()
border.top = xlwt.Borders.THIN
border.bottom = xlwt.Borders.THIN
border.left = xlwt.Borders.THIN
border.right= xlwt.Borders.THIN

pattern1 = xlwt.Pattern()
pattern1.pattern = xlwt.Pattern.SOLID_PATTERN
pattern1.pattern_fore_colour = 22

alignment = xlwt.Alignment()
alignment.vert = 0x00
alignment.horz = 0x00

style.font = font_label
style.alignment = alignment
style.borders = border
style1.font = font
style1.pattern = pattern1
style1.borders = border
style2.font = font
style2.borders = border
#set the colomn wideth
sheet1 = wb.add_sheet('outcome',cell_overwrite_ok=True)
sheet1.col(0).width = 256*35
sheet1.col(1).width = 256*18
sheet1.col(2).width = 256*18
sheet1.col(3).width = 256*17
sheet1.col(4).width = 256*17
sheet1.col(5).width = 256*17
# write the labels
sheet1.write(0,0,'Exam Date(timeslot)',style)
sheet1.write(0,1,'Start Time',style)
sheet1.write(0,2,'End Time',style)
sheet1.write(0,3,'Course Code',style)
sheet1.write(0,4,'Capacity',style)
sheet1.write(0,5,'Room Number',style)
wb.save("test.xlsx")

starttime_dic={0:"8:30",1:"13:30",2:"18:30",}
line = 1# this also represents the index of an exam
for j in Timeslots :
    for i in Courses : 
        if start_time[i,j].X > 0.5 :
            #check whether it is a odd day
            if (j//3+1)%2 == 0 :#even day
                sheet1.write(line,0,"Day"+ str(j//3+1),style1)
                sheet1.write(line,1,starttime_dic.get(j%3),style1)
                sheet1.write(line,3,i,style1)
                # get the end time of the exams
                start_hour,start_minute = starttime_dic.get(j%3).split(":")
                end_minute = (int(start_minute) + int(Duration.get(i)))%60
                end_hour = (int(start_minute) + int(Duration.get(i)))//60 + int(start_hour)
                if end_minute == 0 : 
                    sheet1.write(line,2,str(end_hour)+":00",style1)
                else :
                    sheet1.write(line,2,str(end_hour)+":30",style1)
                #sheet1.write(line,4,Duration.get(i),style1) #show the duration if needed
                sheet1.write(line,4,CourseCapacity[i],style1) #show the capacity if needed
                sheet1.write(line,5,((CourseCapacity[i]-0.001)//30+1),style1) #show the capacity if needed
            else :#odd day
                sheet1.write(line,0,"Day"+ str(j//3+1),style2)
                sheet1.write(line,1,starttime_dic.get(j%3),style2)
                sheet1.write(line,3,i,style2)
                # get the end time of the exams
                start_hour,start_minute = starttime_dic.get(j%3).split(":")
                end_minute = (int(start_minute)+ int(Duration.get(i)))%60
                end_hour = (int(start_minute)+ int(Duration.get(i)))//60 + int(start_hour)
                if end_minute == 0 : 
                    sheet1.write(line,2,str(end_hour)+":00",style2)
                else :
                    sheet1.write(line,2,str(end_hour)+":30",style2)
                #sheet1.write(line,4,Duration.get(i),style2) #show the duration if needed
                sheet1.write(line,4,CourseCapacity[i],style2) #show the capacity if needed
                sheet1.write(line,5,((CourseCapacity[i]-0.001)//30+1),style2) #show the capacity if needed
            line += 1
wb.save("test.xlsx")

#this sheet2 shows the students' number in each slot

sheet2 = wb.add_sheet('statistics data',cell_overwrite_ok=True)

sheet2.col(0).width = 256*15
sheet2.col(1).width = 256*20
sheet2.col(2).width = 256*35
sheet2.col(3).width = 256*35

sheet2.write(0,0,'Timeslot',style)
sheet2.write(0,1,'Number of exams',style)
sheet2.write(0,2,'Total number of students',style)
sheet2.write(0,3,'maximum number in the slot',style)

wb.save("test.xlsx")

for j in Timeslots :
    sheet2.write(j+1,0,j+1,style2)
    count = 0
    studentNum = 0
    temp = 0
    for i in Courses :
        count += start_time[i,j].X
    sheet2.write(j+1,1,count,style2)
    for i in Courses :
        studentNum += start_time[i,j].X*CourseCapacity[i]
    sheet2.write(j+1,2,studentNum,style2)
    for i in Courses :
        if start_time[i,j].X*CourseCapacity[i] > temp :
            temp = int(start_time[i,j].X*CourseCapacity[i])
    sheet2.write(j+1,3,temp,style2)

wb.save("test.xlsx")
